# P&L Curves Comparison

This notebook visualizes and compares cumulative P&L curves across multiple backtest runs.

## Features

- Overlay multiple P&L curves on a single plot for easy comparison
- Interactive plotly charts with hover tooltips and legend controls
- Fast metadata-based filtering before loading full data
- Color-coded traces for distinguishing different runs

## Usage

1. Configure stock symbol and paths
2. Scan available runs to see metadata
3. Filter runs using the summary DataFrame
4. Load selected runs
5. Visualize P&L curves

Use this notebook to quickly identify which parameter combinations lead to better performance profiles.

In [ ]:
# Import required libraries
import sys
from pathlib import Path

# Add analysis folder to path for imports
# Use the project root to construct the analysis path reliably
project_root = Path.cwd()
while project_root.name != 'quiescence' and project_root.parent != project_root:
    project_root = project_root.parent
analysis_path = project_root / 'analysis'

if str(analysis_path) not in sys.path:
    sys.path.insert(0, str(analysis_path))

import pandas as pd
import plotly.graph_objects as go

# Import custom utility functions from analysis folder
from utilities import (
    scan_backtest_runs, 
    load_run_data, 
    create_runs_summary_dataframe, 
    convert_utc_to_ny,
    build_cumulative_pnl_from_positions
)


## 1. Configuration

Set the stock symbol and paths for analysis.

In [ ]:
# === CONFIGURATION ===
stock_symbol = "MSFT"

# Project paths
STORAGE_ROOT = Path("/data/quiescence/")
BACKTEST_ROOT = STORAGE_ROOT / "backtest"

print(f"Analyzing stock: {stock_symbol}")
print(f"Backtest runs root: {BACKTEST_ROOT}")

## 2. Scan and Filter Runs

First, scan all available runs and create a summary DataFrame for filtering.

In [ ]:
# Scan all runs for the configured stock (fast - metadata only)
runs_metadata = scan_backtest_runs(BACKTEST_ROOT, stock_symbol)

# Create summary DataFrame for easy filtering and comparison
df_summary = create_runs_summary_dataframe(runs_metadata)

print(f"\nFound {len(df_summary)} runs for {stock_symbol}\n")
print(df_summary.to_string(index=False))

## 3. Select Runs to Load

Choose which runs to analyze. You can load all, specific runs by number, or filter by parameters.

In [ ]:
# Option 1: Load ALL runs
# runs_data = [load_run_data(run) for run in runs_metadata]

# Option 2: Load specific runs by Run number
selected_run_numbers = [11, 19]
runs_data = [load_run_data(runs_metadata[run_num - 1]) for run_num in selected_run_numbers if 0 < run_num <= len(runs_metadata)]

# Option 3: Filter using DataFrame conditions
# Example: Load runs with specific parameter values
# filtered_df = df_summary[
#     (df_summary['Entry P Top'].astype(float) >= 0.95) & 
#     (df_summary['Frequency'] == '1-MINUTE')
# ]
# selected_indices = filtered_df['Run'].values - 1
# runs_data = [load_run_data(runs_metadata[i]) for i in selected_indices]

print(f"Loaded {len(runs_data)} runs")

## 5. Visualize P&L Curves

Create an interactive plot overlaying all selected runs' P&L curves.

In [ ]:
# Create figure
fig = go.Figure()

# Add each run's P&L curve to the same plot
for i, run in enumerate(runs_data):
    # Extract frequency from bar_type
    bar_type = run.get('bar_type', '')
    if bar_type:
        parts = bar_type.split('-')
        frequency = f"{parts[1]}{parts[2].lower()[0:3]}" if len(parts) >= 3 else 'Unknown'
    else:
        frequency = 'Unknown'
    
    positions = run["positions_report"]
    
    # Build cumulative P&L from positions
    cumulative_pnl = build_cumulative_pnl_from_positions(positions)
    
    # Create a descriptive label for this run
    max_bar_pos = run.get('max_position_bars', 'N/A')
    label = f"{frequency} ({max_bar_pos})"
    
    # Add the P&L curve using cumulative P&L index (timestamps)
    fig.add_trace(go.Scatter(
        x=cumulative_pnl.index, 
        y=cumulative_pnl.values, 
        mode='lines', 
        name=label,
        hovertemplate='<b>' + label + '</b><br>Date: %{x}<br>P&L: $%{y:.2f}<extra></extra>'
    ))

# Update layout
fig.update_layout(
    height=600,
    width=1200,
    title=f'P&L Curves Comparison for {stock_symbol}',
    xaxis_title='Date',
    yaxis_title='Realized P&L ($)',
    hovermode='x unified',
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)

fig.show()

## 6. Summary Statistics

Display final P&L values for each run to quickly identify top performers.

In [ ]:
# Calculate and display final P&L for each run
summary_stats = []

for i, run in enumerate(runs_data):
    positions = run['positions_report']
    
    # Extract frequency
    bar_type = run.get('bar_type', '')
    if bar_type:
        parts = bar_type.split('-')
        frequency = f"{parts[1]}-{parts[2]}" if len(parts) >= 3 else 'Unknown'
    else:
        frequency = 'Unknown'
    
    # Calculate final P&L
    cumulative_pnl = build_cumulative_pnl_from_positions(positions)
    final_pnl = cumulative_pnl.iloc[-1] if len(cumulative_pnl) > 0 else 0
    
    # Calculate max drawdown
    running_max = cumulative_pnl.expanding().max()
    drawdown = cumulative_pnl - running_max
    max_drawdown = drawdown.min()
    
    summary_stats.append({
        'Run': i + 1,
        'Frequency': frequency,
        'Max Pos Bars': run.get('max_position_bars', 'N/A'),
        'Final P&L ($)': f"{final_pnl:.2f}",
        'Max Drawdown ($)': f"{max_drawdown:.2f}",
        'Total Trades': len(positions)
    })

df_stats = pd.DataFrame(summary_stats)
print("\n" + "="*80)
print("P&L SUMMARY STATISTICS")
print("="*80 + "\n")
print(df_stats.to_string(index=False))

## 7. Detailed Position Comparison

Compare positions side-by-side to identify timing and P&L differences between runs.

In [ ]:
# Compare positions between the two loaded runs
if len(runs_data) == 2:
    run1 = runs_data[0]
    run2 = runs_data[1]
    
    # Get positions reports
    pos1 = run1['positions_report'].copy()
    pos2 = run2['positions_report'].copy()
    
    # Extract frequency labels
    bar_type1 = run1.get('bar_type', '')
    bar_type2 = run2.get('bar_type', '')
    parts1 = bar_type1.split('-')
    parts2 = bar_type2.split('-')
    freq1 = f"{parts1[1]}{parts1[2].lower()}" if len(parts1) >= 3 else 'Run1'
    freq2 = f"{parts2[1]}{parts2[2].lower()}" if len(parts2) >= 3 else 'Run2'
    
    # Convert timestamps and clean P&L
    for pos, freq in [(pos1, freq1), (pos2, freq2)]:
        # Convert timestamps to datetime
        pos['entry_time'] = pd.to_datetime(pos['ts_opened'], unit='ns').apply(
            lambda x: convert_utc_to_ny(x.timestamp())
        )
        pos['exit_time'] = pd.to_datetime(pos['ts_closed'], unit='ns').apply(
            lambda x: convert_utc_to_ny(x.timestamp())
        )
        
        # Clean realized P&L
        if pos['realized_pnl'].dtype == 'object':
            pos['pnl'] = pos['realized_pnl'].str.replace(' USD', '').astype(float)
        else:
            pos['pnl'] = pos['realized_pnl']
        
        # Calculate duration in minutes
        pos['duration_min'] = (pos['exit_time'] - pos['entry_time']).dt.total_seconds() / 60
    
    # Create comparison DataFrame
    print(f"\n{'='*100}")
    print(f"POSITION-BY-POSITION COMPARISON: {freq1} vs {freq2}")
    print(f"{'='*100}\n")
    
    print(f"Total positions - {freq1}: {len(pos1)}, {freq2}: {len(pos2)}")
    print(f"Total P&L - {freq1}: ${pos1['pnl'].sum():.2f}, {freq2}: ${pos2['pnl'].sum():.2f}")
    print(f"Difference: ${abs(pos1['pnl'].sum() - pos2['pnl'].sum()):.2f}\n")
    
    # Select key columns for comparison
    cols_to_show = ['entry_time', 'exit_time', 'duration_min', 'entry', 'avg_px_open', 
                    'avg_px_close', 'pnl', 'quantity']
    
    # Rename for clarity
    pos1_display = pos1[cols_to_show].copy()
    pos2_display = pos2[cols_to_show].copy()
    
    pos1_display['run'] = freq1
    pos2_display['run'] = freq2
    
    # Combine and sort by entry time
    combined = pd.concat([pos1_display, pos2_display])
    combined = combined.sort_values('entry_time').reset_index(drop=True)
    
    # Display first 20 positions
    print("First 20 positions (interleaved by entry time):")
    print(combined.head(20).to_string(index=False))
    
    # Calculate position-level statistics
    stats_comparison = pd.DataFrame({
        'Metric': ['Avg P&L per trade', 'Win rate', 'Avg duration (min)', 
                   'Avg winning trade', 'Avg losing trade', 'Largest win', 'Largest loss'],
        freq1: [
            f"${pos1['pnl'].mean():.2f}",
            f"{(pos1['pnl'] > 0).sum() / len(pos1) * 100:.1f}%",
            f"{pos1['duration_min'].mean():.1f}",
            f"${pos1[pos1['pnl'] > 0]['pnl'].mean():.2f}",
            f"${pos1[pos1['pnl'] < 0]['pnl'].mean():.2f}",
            f"${pos1['pnl'].max():.2f}",
            f"${pos1['pnl'].min():.2f}"
        ],
        freq2: [
            f"${pos2['pnl'].mean():.2f}",
            f"{(pos2['pnl'] > 0).sum() / len(pos2) * 100:.1f}%",
            f"{pos2['duration_min'].mean():.1f}",
            f"${pos2[pos2['pnl'] > 0]['pnl'].mean():.2f}",
            f"${pos2[pos2['pnl'] < 0]['pnl'].mean():.2f}",
            f"${pos2['pnl'].max():.2f}",
            f"${pos2['pnl'].min():.2f}"
        ]
    })
    
    print(f"\n\n{'='*80}")
    print("STATISTICAL COMPARISON")
    print(f"{'='*80}\n")
    print(stats_comparison.to_string(index=False))
else:
    print("Please load exactly 2 runs for comparison (modify selected_run_numbers in cell above)")

### 7.1 Entry/Exit Time Alignment Analysis

Check if positions are entering/exiting at exactly the same times or if there's a systematic offset.

In [ ]:
# Analyze timing alignment between the two runs
if len(runs_data) == 2:
    # Sort both by entry time
    pos1_sorted = pos1.sort_values('entry_time').reset_index(drop=True)
    pos2_sorted = pos2.sort_values('entry_time').reset_index(drop=True)
    
    # Try to match positions by proximity (within 5 minutes)
    matches = []
    unmatched_1 = []
    unmatched_2 = set(range(len(pos2_sorted)))
    
    for i, row1 in pos1_sorted.iterrows():
        # Find closest entry in pos2 within 5 minutes
        time_diffs = abs((pos2_sorted['entry_time'] - row1['entry_time']).dt.total_seconds())
        min_diff_idx = time_diffs.idxmin()
        min_diff_seconds = time_diffs[min_diff_idx]
        
        if min_diff_seconds <= 300:  # Within 5 minutes
            matches.append({
                'pos_num': i + 1,
                f'{freq1}_entry': row1['entry_time'],
                f'{freq2}_entry': pos2_sorted.loc[min_diff_idx, 'entry_time'],
                'entry_diff_sec': (pos2_sorted.loc[min_diff_idx, 'entry_time'] - row1['entry_time']).total_seconds(),
                f'{freq1}_exit': row1['exit_time'],
                f'{freq2}_exit': pos2_sorted.loc[min_diff_idx, 'exit_time'],
                'exit_diff_sec': (pos2_sorted.loc[min_diff_idx, 'exit_time'] - row1['exit_time']).total_seconds(),
                f'{freq1}_pnl': row1['pnl'],
                f'{freq2}_pnl': pos2_sorted.loc[min_diff_idx, 'pnl'],
                'pnl_diff': pos2_sorted.loc[min_diff_idx, 'pnl'] - row1['pnl']
            })
            unmatched_2.discard(min_diff_idx)
        else:
            unmatched_1.append(i)
    
    df_matches = pd.DataFrame(matches)
    
    print(f"\n{'='*100}")
    print("TIMING ALIGNMENT ANALYSIS")
    print(f"{'='*100}\n")
    print(f"Matched positions: {len(matches)}")
    print(f"Unmatched in {freq1}: {len(unmatched_1)}")
    print(f"Unmatched in {freq2}: {len(unmatched_2)}\n")
    
    if len(matches) > 0:
        print("Entry timing differences (seconds):")
        print(f"  Mean: {df_matches['entry_diff_sec'].mean():.2f}s")
        print(f"  Median: {df_matches['entry_diff_sec'].median():.2f}s")
        print(f"  Std: {df_matches['entry_diff_sec'].std():.2f}s")
        print(f"  Range: [{df_matches['entry_diff_sec'].min():.2f}s, {df_matches['entry_diff_sec'].max():.2f}s]")
        
        print("\nExit timing differences (seconds):")
        print(f"  Mean: {df_matches['exit_diff_sec'].mean():.2f}s")
        print(f"  Median: {df_matches['exit_diff_sec'].median():.2f}s")
        print(f"  Std: {df_matches['exit_diff_sec'].std():.2f}s")
        print(f"  Range: [{df_matches['exit_diff_sec'].min():.2f}s, {df_matches['exit_diff_sec'].max():.2f}s]")
        
        print("\nP&L differences (matched positions only):")
        print(f"  Mean: ${df_matches['pnl_diff'].mean():.2f}")
        print(f"  Median: ${df_matches['pnl_diff'].median():.2f}")
        print(f"  Total: ${df_matches['pnl_diff'].sum():.2f}")
        print(f"  Range: [${df_matches['pnl_diff'].min():.2f}, ${df_matches['pnl_diff'].max():.2f}]")
        
        # Show positions with largest timing differences
        print("\n\nPositions with largest entry timing differences:")
        timing_diffs = df_matches.reindex(df_matches['entry_diff_sec'].abs().nlargest(5).index)
        print(timing_diffs[[f'{freq1}_entry', f'{freq2}_entry', 'entry_diff_sec', 
                           f'{freq1}_pnl', f'{freq2}_pnl', 'pnl_diff']].to_string(index=False))
        
        # Show positions with largest P&L differences
        print("\n\nPositions with largest P&L differences:")
        pnl_diffs = df_matches.reindex(df_matches['pnl_diff'].abs().nlargest(5).index)
        print(pnl_diffs[[f'{freq1}_entry', f'{freq2}_entry', 'entry_diff_sec',
                        f'{freq1}_pnl', f'{freq2}_pnl', 'pnl_diff']].to_string(index=False))
else:
    print("Please load exactly 2 runs for comparison")

### 7.2 Bar Data Comparison

Compare the underlying bar data to see if OHLCV values differ between 1-MINUTE and 60-SECOND bars.

In [ ]:
# Compare bar data between the two runs
if len(runs_data) == 2:
    df1 = run1['data'].copy()
    df2 = run2['data'].copy()
    
    # Ensure we have datetime columns
    if 'date' in df1.columns:
        df1 = df1.rename(columns={'date': 'bar_close_local_datetime'})
    if 'date' in df2.columns:
        df2 = df2.rename(columns={'date': 'bar_close_local_datetime'})
    
    # Convert to datetime if needed
    if df1['bar_close_local_datetime'].dtype == 'object':
        df1['bar_close_local_datetime'] = pd.to_datetime(df1['bar_close_local_datetime'])
    if df2['bar_close_local_datetime'].dtype == 'object':
        df2['bar_close_local_datetime'] = pd.to_datetime(df2['bar_close_local_datetime'])
    
    print(f"\n{'='*100}")
    print("BAR DATA COMPARISON")
    print(f"{'='*100}\n")
    print(f"Total bars - {freq1}: {len(df1)}, {freq2}: {len(df2)}")
    
    # First, check exact timestamp alignment (not rounded)
    df1['ts_index'] = range(len(df1))
    df2['ts_index'] = range(len(df2))
    
    # Compare first 10 bar timestamps exactly
    print("\n\nFirst 10 bar timestamps (exact comparison):")
    ts_comparison = pd.DataFrame({
        f'{freq1}_timestamp': df1['bar_close_local_datetime'].head(10),
        f'{freq2}_timestamp': df2['bar_close_local_datetime'].head(10),
        'diff_seconds': (df2['bar_close_local_datetime'].head(10).values - df1['bar_close_local_datetime'].head(10).values) / pd.Timedelta(seconds=1)
    })
    print(ts_comparison.to_string(index=True))
    
    # Calculate timestamp differences across all bars
    if len(df1) == len(df2):
        time_diffs = (df2['bar_close_local_datetime'] - df1['bar_close_local_datetime']).dt.total_seconds()
        unique_diffs = time_diffs.unique()
        
        print(f"\n\nTimestamp offset analysis:")
        print(f"  Unique offsets found: {len(unique_diffs)}")
        if len(unique_diffs) <= 10:
            print(f"  Offset values (seconds): {sorted(unique_diffs)}")
        else:
            print(f"  Offset range: [{time_diffs.min():.2f}s, {time_diffs.max():.2f}s]")
            print(f"  Mean offset: {time_diffs.mean():.2f}s")
            print(f"  Median offset: {time_diffs.median():.2f}s")
    
    # Try to merge on timestamp (rounded to minute for alignment)
    df1_copy = df1.copy()
    df2_copy = df2.copy()
    
    # Round to nearest minute for joining (using 'min' instead of deprecated 'T')
    df1_copy['ts_minute'] = df1_copy['bar_close_local_datetime'].dt.floor('min')
    df2_copy['ts_minute'] = df2_copy['bar_close_local_datetime'].dt.floor('min')
    
    # Merge on rounded minute
    merged = df1_copy.merge(
        df2_copy, 
        on='ts_minute', 
        how='outer', 
        suffixes=(f'_{freq1}', f'_{freq2}'),
        indicator=True
    )
    
    # Count alignment
    both = (merged['_merge'] == 'both').sum()
    only_1 = (merged['_merge'] == 'left_only').sum()
    only_2 = (merged['_merge'] == 'right_only').sum()
    
    print(f"\n\nBar alignment (rounded to nearest minute):")
    print(f"  Bars in both: {both}")
    print(f"  Only in {freq1}: {only_1}")
    print(f"  Only in {freq2}: {only_2}")
    
    # For bars that exist in both, compare OHLCV values
    if both > 0:
        matched = merged[merged['_merge'] == 'both'].copy()
        
        # Calculate differences in OHLC
        for field in ['open', 'high', 'low', 'close']:
            if f'{field}_{freq1}' in matched.columns and f'{field}_{freq2}' in matched.columns:
                matched[f'{field}_diff'] = matched[f'{field}_{freq2}'] - matched[f'{field}_{freq1}']
        
        print(f"\n\nPrice differences for matched bars (n={both}):")
        for field in ['open', 'high', 'low', 'close']:
            diff_col = f'{field}_diff'
            if diff_col in matched.columns:
                diffs = matched[diff_col].dropna()
                non_zero = (diffs != 0).sum()
                if len(diffs) > 0:
                    print(f"\n  {field.upper()}:")
                    print(f"    Bars with differences: {non_zero} ({non_zero/len(diffs)*100:.1f}%)")
                    if non_zero > 0:
                        print(f"    Mean diff: ${diffs.mean():.4f}")
                        print(f"    Max diff: ${diffs.max():.4f}")
                        print(f"    Min diff: ${diffs.min():.4f}")
        
        # Show sample of bars with differences
        print("\n\nSample of bars with price differences:")
        diffs_exist = False
        for field in ['open', 'high', 'low', 'close']:
            diff_col = f'{field}_diff'
            if diff_col in matched.columns:
                if (matched[diff_col] != 0).any():
                    diffs_exist = True
                    break
        
        if diffs_exist:
            # Find bars with any price difference
            matched['has_diff'] = False
            for field in ['open', 'high', 'low', 'close']:
                diff_col = f'{field}_diff'
                if diff_col in matched.columns:
                    matched['has_diff'] |= (matched[diff_col] != 0)
            
            diff_bars = matched[matched['has_diff']].head(10)
            cols_to_show = ['ts_minute']
            for field in ['open', 'high', 'low', 'close']:
                if f'{field}_{freq1}' in diff_bars.columns:
                    cols_to_show.extend([f'{field}_{freq1}', f'{field}_{freq2}', f'{field}_diff'])
            
            print(diff_bars[cols_to_show].to_string(index=False))
        else:
            print("No price differences found in matched bars!")
            print("\n⚠️  CONCLUSION: Bar data is IDENTICAL between runs.")
            print("   P&L differences must come from timing offsets in bar timestamps,")
            print("   causing signals to trigger at slightly different times.")
    
else:
    print("Please load exactly 2 runs for comparison")

### 7.3 Root Cause Analysis

Since bar OHLCV values are identical, the P&L differences must come from timing offsets. This analysis identifies when and how the timing differences affect trading decisions.

In [ ]:
# Root cause: identify how timestamp offsets affect trading decisions
if len(runs_data) == 2:
    print(f"\n{'='*100}")
    print("ROOT CAUSE ANALYSIS: HOW TIMING OFFSETS AFFECT TRADES")
    print(f"{'='*100}\n")
    
    # Check if timestamps differ and by how much
    df1 = run1['data'].copy()
    df2 = run2['data'].copy()
    
    if 'date' in df1.columns:
        df1['ts'] = pd.to_datetime(df1['date'])
    else:
        df1['ts'] = pd.to_datetime(df1['bar_close_local_datetime'])
    
    if 'date' in df2.columns:
        df2['ts'] = pd.to_datetime(df2['date'])
    else:
        df2['ts'] = pd.to_datetime(df2['bar_close_local_datetime'])
    
    if len(df1) == len(df2):
        # Calculate the systematic offset
        offset_seconds = (df2['ts'].iloc[0] - df1['ts'].iloc[0]).total_seconds()
        
        print(f"Systematic timestamp offset: {offset_seconds} seconds")
        print(f"  {freq1} first bar: {df1['ts'].iloc[0]}")
        print(f"  {freq2} first bar: {df2['ts'].iloc[0]}")
        
        if abs(offset_seconds) > 0:
            print(f"\n💡 KEY FINDING:")
            print(f"   Bar timestamps are offset by {offset_seconds} seconds.")
            print(f"   This means 1-MINUTE bars close on the minute boundary (e.g., 09:31:00),")
            print(f"   while 60-SECOND bars close {offset_seconds}s {'after' if offset_seconds > 0 else 'before'} the minute.")
            print(f"\n   Strategy signals computed on these bars will trigger at different times,")
            print(f"   leading to different entry/exit prices and ultimately different P&L.")
            
            # Check if matched positions have timing that correlates with this offset
            if 'df_matches' in locals() and len(df_matches) > 0:
                median_entry_diff = df_matches['entry_diff_sec'].median()
                print(f"\n   Median position entry difference: {median_entry_diff:.1f} seconds")
                print(f"   This aligns with the {abs(offset_seconds):.1f}s bar timestamp offset!")
        else:
            print(f"\n❓ Timestamps are identical, yet positions differ.")
            print(f"   Investigating other factors...")
            
            # Check for differences in position entry logic
            print(f"\n   Possible causes:")
            print(f"   - Floating point precision in signal calculations")
            print(f"   - Race conditions in bar processing order")
            print(f"   - Different Python/NautilusTrader versions")
    else:
        print(f"Bar counts differ: {len(df1)} vs {len(df2)}")
        print(f"This suggests different data ranges or filtering.")
    
else:
    print("Please load exactly 2 runs for comparison")

### 7.4 Close Price Comparison at Exit Times

Check if the bar close prices differ at position exit times - this would directly explain P&L differences.

In [ ]:
runs_data[0]['data'].head()


In [ ]:
# Compare close prices at position exit times
if len(runs_data) == 2 and 'df_matches' in locals() and len(df_matches) > 0:
    print(f"\n{'='*100}")
    print("CLOSE PRICE COMPARISON AT EXIT TIMES")
    print(f"{'='*100}\n")
    
    # Get the bar data
    df1 = run1['data'].copy()
    df2 = run2['data'].copy()
    
    # Ensure datetime columns
    if 'date' in df1.columns:
        df1['bar_close_local_datetime'] = pd.to_datetime(df1['date'])
    else:
        df1['bar_close_local_datetime'] = pd.to_datetime(df1['bar_close_local_datetime'])
    
    if 'date' in df2.columns:
        df2['bar_close_local_datetime'] = pd.to_datetime(df2['date'])
    else:
        df2['bar_close_local_datetime'] = pd.to_datetime(df2['bar_close_local_datetime'])
    
    # Determine the close price column name
    close_col1 = 'close_price' if 'close_price' in df1.columns else 'close'
    close_col2 = 'close_price' if 'close_price' in df2.columns else 'close'
    
    # Set index for fast lookup
    df1_indexed = df1.set_index('bar_close_local_datetime')
    df2_indexed = df2.set_index('bar_close_local_datetime')
    
    # Localize bar timestamps to NY timezone if they're naive (to match position timestamps)
    if df1_indexed.index.tz is None:
        df1_indexed.index = df1_indexed.index.tz_localize('America/New_York')
    if df2_indexed.index.tz is None:
        df2_indexed.index = df2_indexed.index.tz_localize('America/New_York')
    
    # For each matched position, find the close prices at entry and exit
    price_comparisons = []
    
    for idx, row in df_matches.head(20).iterrows():  # Check first 20 matched positions
        entry1 = row[f'{freq1}_entry']
        entry2 = row[f'{freq2}_entry']
        exit1 = row[f'{freq1}_exit']
        exit2 = row[f'{freq2}_exit']
        
        # Find close prices at exit times (or nearest bar)
        try:
            # Get close price at or before exit time (forward fill)
            close1_exit = df1_indexed[close_col1].asof(exit1)
            close2_exit = df2_indexed[close_col2].asof(exit2)
            
            # Get close price at or before entry time
            close1_entry = df1_indexed[close_col1].asof(entry1)
            close2_entry = df2_indexed[close_col2].asof(entry2)
            
            price_comparisons.append({
                'pos_num': row['pos_num'],
                f'{freq1}_exit_time': exit1,
                f'{freq2}_exit_time': exit2,
                'exit_time_diff_sec': row['exit_diff_sec'],
                f'{freq1}_close_at_exit': close1_exit,
                f'{freq2}_close_at_exit': close2_exit,
                'close_price_diff': close2_exit - close1_exit,
                f'{freq1}_pnl': row[f'{freq1}_pnl'],
                f'{freq2}_pnl': row[f'{freq2}_pnl'],
                'pnl_diff': row['pnl_diff']
            })
        except Exception as e:
            print(f"Warning: Could not find close prices for position {row['pos_num']}: {e}")
            continue
    
    df_price_comp = pd.DataFrame(price_comparisons)
    
    if len(df_price_comp) > 0:
        print(f"Analyzed {len(df_price_comp)} matched positions\n")
        
        # Count positions with close price differences
        close_price_diffs = df_price_comp['close_price_diff'].dropna()
        non_zero_diffs = (close_price_diffs != 0).sum()
        
        print(f"Close price differences at exit times:")
        print(f"  Positions with different close prices: {non_zero_diffs} ({non_zero_diffs/len(close_price_diffs)*100:.1f}%)")
        
        if non_zero_diffs > 0:
            print(f"  Mean price difference: ${close_price_diffs.mean():.4f}")
            print(f"  Max price difference: ${close_price_diffs.max():.4f}")
            print(f"  Min price difference: ${close_price_diffs.min():.4f}")
            print(f"  Std price difference: ${close_price_diffs.std():.4f}")
            
            print("\n\n💡 KEY FINDING:")
            print(f"   Close prices differ at exit times for {non_zero_diffs} positions!")
            print(f"   Even though OHLCV data looks identical when aligned by minute,")
            print(f"   the timing offset means positions exit on different bars")
            print(f"   with different close prices, causing P&L to diverge.")
            
            # Show positions with largest close price differences
            print("\n\nPositions with largest close price differences at exit:")
            largest_price_diffs = df_price_comp.reindex(df_price_comp['close_price_diff'].abs().nlargest(5).index)
            display_cols = [f'{freq1}_exit_time', f'{freq2}_exit_time', 'exit_time_diff_sec',
                          f'{freq1}_close_at_exit', f'{freq2}_close_at_exit', 'close_price_diff',
                          f'{freq1}_pnl', f'{freq2}_pnl', 'pnl_diff']
            print(largest_price_diffs[display_cols].to_string(index=False))
            
            # Correlation analysis
            if len(close_price_diffs) > 2:
                correlation = df_price_comp[['close_price_diff', 'pnl_diff']].corr().iloc[0, 1]
                print(f"\n\nCorrelation between close price diff and P&L diff: {correlation:.3f}")
                if abs(correlation) > 0.5:
                    print(f"   Strong correlation! Close price differences ARE causing P&L differences.")
                else:
                    print(f"   Weak correlation. Other factors may also be contributing.")
        else:
            print("   All close prices are identical at exit times.")
            print("   P&L differences must come from other factors (entry prices, fees, etc.)")
        
        # Show sample of all comparisons
        print("\n\nSample of position exit price comparisons (first 10):")
        display_cols = [f'{freq1}_exit_time', f'{freq2}_exit_time', 
                       f'{freq1}_close_at_exit', f'{freq2}_close_at_exit', 'close_price_diff',
                       f'{freq1}_pnl', f'{freq2}_pnl', 'pnl_diff']
        print(df_price_comp[display_cols].head(10).to_string(index=False))
    else:
        print("Could not extract close price data for comparison.")
        
else:
    print("Need 2 runs with matched positions to compare close prices.")

### 7.5 Entry and Exit Price Comparison

Compare actual entry (avg_px_open) and exit (avg_px_close) prices from position reports to identify where P&L differences originate.

In [ ]:
# Compare entry and exit prices from position reports
if len(runs_data) == 2 and 'df_matches' in locals() and len(df_matches) > 0:
    print(f"\n{'='*100}")
    print("ENTRY AND EXIT PRICE COMPARISON (FROM POSITION REPORTS)")
    print(f"{'='*100}\n")
    
    # Build detailed comparison using position data
    pos1 = runs_data[0]['positions_report'].copy()
    pos2 = runs_data[1]['positions_report'].copy()
    
    # Convert timestamps for matching
    for pos in [pos1, pos2]:
        pos['entry_time'] = pd.to_datetime(pos['ts_opened'], unit='ns').apply(
            lambda x: convert_utc_to_ny(x.timestamp())
        )
        pos['exit_time'] = pd.to_datetime(pos['ts_closed'], unit='ns').apply(
            lambda x: convert_utc_to_ny(x.timestamp())
        )
    
    # Sort by entry time
    pos1_sorted = pos1.sort_values('entry_time').reset_index(drop=True)
    pos2_sorted = pos2.sort_values('entry_time').reset_index(drop=True)
    
    # Match positions (same logic as before)
    price_matches = []
    
    for i, row1 in pos1_sorted.iterrows():
        # Find closest entry in pos2 within 5 minutes
        time_diffs = abs((pos2_sorted['entry_time'] - row1['entry_time']).dt.total_seconds())
        min_diff_idx = time_diffs.idxmin()
        min_diff_seconds = time_diffs[min_diff_idx]
        
        if min_diff_seconds <= 300:  # Within 5 minutes
            row2 = pos2_sorted.loc[min_diff_idx]
            
            # Extract P&L values
            if isinstance(row1['realized_pnl'], str):
                pnl1 = float(row1['realized_pnl'].replace(' USD', ''))
            else:
                pnl1 = float(row1['realized_pnl'])
            
            if isinstance(row2['realized_pnl'], str):
                pnl2 = float(row2['realized_pnl'].replace(' USD', ''))
            else:
                pnl2 = float(row2['realized_pnl'])
            
            price_matches.append({
                'pos_num': i + 1,
                f'{freq1}_entry_time': row1['entry_time'],
                f'{freq2}_entry_time': row2['entry_time'],
                'entry_time_diff_sec': (row2['entry_time'] - row1['entry_time']).total_seconds(),
                f'{freq1}_avg_px_open': float(row1['avg_px_open']),
                f'{freq2}_avg_px_open': float(row2['avg_px_open']),
                'entry_price_diff': float(row2['avg_px_open']) - float(row1['avg_px_open']),
                f'{freq1}_avg_px_close': float(row1['avg_px_close']),
                f'{freq2}_avg_px_close': float(row2['avg_px_close']),
                'exit_price_diff': float(row2['avg_px_close']) - float(row1['avg_px_close']),
                'quantity': int(row1['quantity']),
                f'{freq1}_pnl': pnl1,
                f'{freq2}_pnl': pnl2,
                'pnl_diff': pnl2 - pnl1
            })
    
    df_price_match = pd.DataFrame(price_matches)
    
    if len(df_price_match) > 0:
        print(f"Analyzed {len(df_price_match)} matched positions\n")
        
        # Entry price analysis
        entry_price_diffs = df_price_match['entry_price_diff']
        non_zero_entry = (entry_price_diffs != 0).sum()
        
        print(f"ENTRY PRICE DIFFERENCES (avg_px_open):")
        print(f"  Positions with different entry prices: {non_zero_entry} ({non_zero_entry/len(entry_price_diffs)*100:.1f}%)")
        
        if non_zero_entry > 0:
            print(f"  Mean difference: ${entry_price_diffs.mean():.6f}")
            print(f"  Median difference: ${entry_price_diffs.median():.6f}")
            print(f"  Max difference: ${entry_price_diffs.max():.6f}")
            print(f"  Min difference: ${entry_price_diffs.min():.6f}")
            print(f"  Std difference: ${entry_price_diffs.std():.6f}")
            
            # Calculate impact on P&L (price diff * quantity)
            df_price_match['entry_price_impact'] = df_price_match['entry_price_diff'] * df_price_match['quantity']
            print(f"\n  Average P&L impact from entry price diff: ${df_price_match['entry_price_impact'].mean():.2f}")
            print(f"  Total P&L impact from entry price diffs: ${df_price_match['entry_price_impact'].sum():.2f}")
        
        # Exit price analysis
        exit_price_diffs = df_price_match['exit_price_diff']
        non_zero_exit = (exit_price_diffs != 0).sum()
        
        print(f"\n\nEXIT PRICE DIFFERENCES (avg_px_close):")
        print(f"  Positions with different exit prices: {non_zero_exit} ({non_zero_exit/len(exit_price_diffs)*100:.1f}%)")
        
        if non_zero_exit > 0:
            print(f"  Mean difference: ${exit_price_diffs.mean():.6f}")
            print(f"  Median difference: ${exit_price_diffs.median():.6f}")
            print(f"  Max difference: ${exit_price_diffs.max():.6f}")
            print(f"  Min difference: ${exit_price_diffs.min():.6f}")
            print(f"  Std difference: ${exit_price_diffs.std():.6f}")
            
            # Calculate impact on P&L
            df_price_match['exit_price_impact'] = df_price_match['exit_price_diff'] * df_price_match['quantity']
            print(f"\n  Average P&L impact from exit price diff: ${df_price_match['exit_price_impact'].mean():.2f}")
            print(f"  Total P&L impact from exit price diffs: ${df_price_match['exit_price_impact'].sum():.2f}")
        
        # Combined analysis
        print(f"\n\n{'='*80}")
        print("💡 ROOT CAUSE SUMMARY")
        print(f"{'='*80}\n")
        
        # Calculate correlation between price differences and P&L differences
        if non_zero_entry > 0 or non_zero_exit > 0:
            df_price_match['total_price_impact'] = df_price_match['entry_price_impact'] + df_price_match['exit_price_impact']
            
            # For LONG positions, P&L = (exit_price - entry_price) * quantity
            # So: pnl_diff = (exit_price2 - entry_price2) - (exit_price1 - entry_price1)
            #              = (exit_price_diff - entry_price_diff) * quantity
            df_price_match['expected_pnl_diff'] = (df_price_match['exit_price_diff'] - df_price_match['entry_price_diff']) * df_price_match['quantity']
            
            print(f"Total P&L difference (actual): ${df_price_match['pnl_diff'].sum():.2f}")
            print(f"Total P&L difference (expected from price diffs): ${df_price_match['expected_pnl_diff'].sum():.2f}")
            print(f"Difference explained: {abs(df_price_match['expected_pnl_diff'].sum() / df_price_match['pnl_diff'].sum() * 100):.1f}%")
            
            # Correlation
            if len(df_price_match) > 2:
                corr_entry = df_price_match[['entry_price_impact', 'pnl_diff']].corr().iloc[0, 1]
                corr_exit = df_price_match[['exit_price_impact', 'pnl_diff']].corr().iloc[0, 1]
                corr_total = df_price_match[['expected_pnl_diff', 'pnl_diff']].corr().iloc[0, 1]
                
                print(f"\nCorrelations with P&L difference:")
                print(f"  Entry price impact: {corr_entry:.3f}")
                print(f"  Exit price impact: {corr_exit:.3f}")
                print(f"  Combined price impact: {corr_total:.3f}")
        
        # Show examples with largest entry price differences
        print(f"\n\nPositions with largest entry price differences:")
        largest_entry_diffs = df_price_match.reindex(df_price_match['entry_price_diff'].abs().nlargest(10).index)
        display_cols = [f'{freq1}_entry_time', f'{freq2}_entry_time', 'entry_time_diff_sec',
                       f'{freq1}_avg_px_open', f'{freq2}_avg_px_open', 'entry_price_diff',
                       'quantity', 'entry_price_impact', f'{freq1}_pnl', f'{freq2}_pnl', 'pnl_diff']
        print(largest_entry_diffs[display_cols].to_string(index=False))
        
        # Show examples with largest exit price differences
        if non_zero_exit > 0:
            print(f"\n\nPositions with largest exit price differences:")
            largest_exit_diffs = df_price_match.reindex(df_price_match['exit_price_diff'].abs().nlargest(10).index)
            display_cols = [f'{freq1}_exit_time', f'{freq2}_exit_time',
                           f'{freq1}_avg_px_close', f'{freq2}_avg_px_close', 'exit_price_diff',
                           'quantity', 'exit_price_impact', f'{freq1}_pnl', f'{freq2}_pnl', 'pnl_diff']
            print(largest_exit_diffs[[col for col in display_cols if col in df_price_match.columns]].to_string(index=False))
        
    else:
        print("Could not match positions for price comparison.")
        
else:
    print("Need 2 runs with matched positions to compare prices.")